<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [1]:
# Clone MesoNet repository
!git clone https://github.com/bf777/MesoNet.git

Cloning into 'MesoNet'...
remote: Enumerating objects: 1150, done.
remote: Counting objects: 100% (603/603), done.
remote: Compressing objects: 100% (461/461), done.
remote: Total 1150 (delta 356), reused 356 (delta 136), pack-reused 547
Receiving objects: 100% (1150/1150), 7.34 MiB | 25.14 MiB/s, done.
Resolving deltas: 100% (740/740), done.


# Prepare inputs

In [2]:
!mkdir /content/mesonet_inputs/
!mkdir /content/mesonet_inputs/pipeline1_2/

In [3]:
%cd MesoNet/

/content/MesoNet


In [4]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3

# Install DeepLabCut for pose estimation
!pip install deeplabcut

     |████████████████████████████████| 13.1 MB 90 kB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 540 kB 5.1 MB/s 
     |████████████████████████████████| 108 kB 66.3 MB/s 
     |████████████████████████████████| 9.5 MB 55.7 MB/s 
     |████████████████████████████████| 296 kB 48.1 MB/s 
     |████████████████████████████████| 177 kB 45.8 MB/s 
     |████████████████████████████████| 29.2 MB 50 kB/s 
     |████████████████████████████████| 37.1 MB 48 kB/s 
     |████████████████████████████████| 352 kB 36.7 MB/s 
     |████████████████████████████████| 546 kB 43.1 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110475 sha256=be8a3bf23c6f889f589898f75329cf4cd86d100d527ee15a8e69667578d2b113
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2
  Attempting

In [5]:
# Install MesoNet
!python setup.py install

# Reinstall OpenCV to address compatibility issue
!pip install opencv-python==4.4.0.46

# Reinstall h5py
!pip install h5py==2.10.0

# pip install mesonet

running install
running bdist_egg
running egg_info
creating mesonet.egg-info
writing mesonet.egg-info/PKG-INFO
writing dependency_links to mesonet.egg-info/dependency_links.txt
writing requirements to mesonet.egg-info/requires.txt
writing top-level names to mesonet.egg-info/top_level.txt
writing manifest file 'mesonet.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'mesonet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mesonet
copying mesonet/gui_train.py -> build/lib/mesonet
copying mesonet/data.py -> build/lib/mesonet
copying mesonet/voxelmorph_align.py -> build/lib/mesonet
copying mesonet/atlas_brain_matching.py -> build/lib/mesonet
copying mesonet/mask_functions.py -> build/lib/mesonet
copying mesonet/gui_start.py -> build/lib/mesonet
copying mesonet/img_augment.py -> build/lib/mesonet
copying mesonet/utils.py -> build/lib/mesonet
copyi

Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

In [6]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
import os
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [7]:
!pip install osfclient

In [8]:
%cd /content/
!osf -p svztu fetch 6_Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

/content
100% 643M/643M [00:05<00:00, 129Mbytes/s]


In [9]:
!unzip -q mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip -d /content/mesonet_inputs

In [10]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_brain_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5

/content
100% 373M/373M [00:03<00:00, 105Mbytes/s]


In [11]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_unet_motif_based_functional_atlas.hdf5 MesoNet/mesonet/models/DongshengXiao_unet_motif_based_functional_atlas.hdf5

/content
100% 373M/373M [00:02<00:00, 154Mbytes/s]


In [12]:
%cd /content/
!osf -p svztu fetch 8_VoxelMorph_model/VoxelMorph_Motif_based_functional_map_model_transformed1000.h5 mesonet_inputs/voxelMorph_Motif_based_functional_map_model_transformed1000.h5

/content
100% 1.51M/1.51M [00:00<00:00, 141Mbytes/s]


# Five ways to use MesoNet

MesoNet can be used through five approaches:
1. **Atlas to brain**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, register an atlas of brain regions to the fixed brain imaging data using affine transformations. This approach is useful if your data has common anatomical landmarks and is the most robust to variations in image quality and orientation within your data.
2. **Brain to atlas**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, the brain imaging data to a fixed atlas of brain regions using affine transformations. This approach is useful if you would like to normalize your brain images to a common template based on anatomical landmarks.
3. **Atlas to brain + sensory maps**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions as well as a set of folders containing functional brain activity for that animal that is consistent across animals, register an atlas of brain regions to the fixed brain imaging data using affine transformations. This approach is useful if you have consistent peaks of functional activity across animals that you would like to use in the alignment processes.
4. **Motif-based functional maps (MBFMs) + U-Net**: Given a pre-trained U-Net model that was trained to associate brain imaging data with atlases of brain regions, predict the locations of brain regions in the data without the use of landmarks. The brain imaging data should be motif-based functional maps (MBFMs) calculated using the associated MATLAB code (using seqNMF). This approach is useful if one wishes to mark functional regions based on more complex features of the data (e.g. a motif-based functional map) than landmarks.
5. **Motif-based functional maps (MBFMs) + Brain-to-atlas + VoxelMorph**: Given a pre-trained VoxelMorph model that was trained to compute a non-linear transformation between a template functional brain atlas and brain image data, predict the locations of brain regions in the data. In particular, this approach can register each input brain image to a user-defined template functional atlas. The brain imaging data should be motif-based functional maps (MBFMs) calculated using the associated MATLAB code (using seqNMF). This approach is useful if your images are consistently oriented and you want to compare the predicted locations of brain regions across different images.

We will now copy over some sample input images for each of these five pipelines from OSF to our inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell.

In [13]:
#@title Run this cell to fetch sample data from OSF

%cd /content/
# Pipeline 1 + 2
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline1_2/0.png mesonet_inputs/pipeline1_2/0.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline1_2/1.png mesonet_inputs/pipeline1_2/1.png

# Pipeline 3
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/tail.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/visual.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/whisker.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/tail.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/visual.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/whisker.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/tail.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/visual.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/whisker.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/1.png mesonet_inputs/pipeline3_sensory/sensory_raw/1.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/2.png mesonet_inputs/pipeline3_sensory/sensory_raw/2.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/3.png mesonet_inputs/pipeline3_sensory/sensory_raw/3.png

# Pipeline 4
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/0.png mesonet_inputs/pipeline4_MBFM-U-Net/0.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/1.png mesonet_inputs/pipeline4_MBFM-U-Net/1.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/2.png mesonet_inputs/pipeline4_MBFM-U-Net/2.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/3.png mesonet_inputs/pipeline4_MBFM-U-Net/3.png

# Pipeline 5
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline5_VoxelMorph/0.png mesonet_inputs/pipeline5_VoxelMorph/0.png
!osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline5_VoxelMorph/1.png mesonet_inputs/pipeline5_VoxelMorph/1.png

/content
100% 6.88k/6.88k [00:00<00:00, 17.3Mbytes/s]
100% 7.04k/7.04k [00:00<00:00, 16.8Mbytes/s]
100% 137k/137k [00:00<00:00, 76.4Mbytes/s]
100% 127k/127k [00:00<00:00, 80.0Mbytes/s]
100% 128k/128k [00:00<00:00, 50.8Mbytes/s]
100% 135k/135k [00:00<00:00, 73.2Mbytes/s]
100% 120k/120k [00:00<00:00, 52.7Mbytes/s]
100% 126k/126k [00:00<00:00, 74.1Mbytes/s]
100% 157k/157k [00:00<00:00, 85.5Mbytes/s]
100% 167k/167k [00:00<00:00, 76.8Mbytes/s]
100% 146k/146k [00:00<00:00, 73.9Mbytes/s]
100% 13.4k/13.4k [00:00<00:00, 31.2Mbytes/s]
100% 13.8k/13.8k [00:00<00:00, 28.0Mbytes/s]
100% 13.8k/13.8k [00:00<00:00, 31.4Mbytes/s]
100% 19.5k/19.5k [00:00<00:00, 34.2Mbytes/s]
100% 21.3k/21.3k [00:00<00:00, 32.6Mbytes/s]
100% 21.0k/21.0k [00:00<00:00, 35.0Mbytes/s]
100% 19.6k/19.6k [00:00<00:00, 34.8Mbytes/s]
100% 6.88k/6.88k [00:00<00:00, 16.6Mbytes/s]


Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of a folder in `mesonet_inputs` containing your input data.

In [20]:
#@title Input information for the model
input_file_name = 'pipeline1_2'  #@param {type: "string"}
input_file_sensory_raw_name = 'sensory_raw'  #@param {type: "string"}
input_file_sensory_maps_name = 'sensory_maps'  #@param {type: "string"}
input_file_MBFM_name = 'pipeline4_MBFM-U-Net'  #@param {type: "string"}
input_file_voxelmorph_name = 'pipeline5_VoxelMorph'  #@param {type: "string"}

output_file_atlas_brain_name = 'mesonet_outputs_atlas_brain'  #@param {type: "string"}
output_file_brain_atlas_name = 'mesonet_outputs_brain_atlas'  #@param {type: "string"}
output_file_sensory_name = 'mesonet_outputs_sensory'  #@param {type: "string"}
output_file_MBFM_U_Net_name = 'mesonet_outputs_MBFM_U_Net'  #@param {type: "string"}
output_file_voxelmorph_name = 'mesonet_outputs_voxelmorph'  #@param {type: "string"}

model_name = 'DongshengXiao_brain_bundary.hdf5' #@param {type: "string"}
u_net_only_model_name = 'DongshengXiao_unet_motif_based_functional_atlas.hdf5'
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}
voxelmorph_model_name = 'voxelMorph_Motif_based_functional_map_model_transformed1000.h5'  #@param {type: "string"}

In [21]:
# Set up filepaths based on your inputs
input_file = os.path.join('/content','mesonet_inputs', input_file_name)
input_file_sensory_raw = os.path.join('/content','mesonet_inputs', 'pipeline3_sensory', input_file_sensory_raw_name)
input_file_sensory_maps = os.path.join('/content','mesonet_inputs', 'pipeline3_sensory', input_file_sensory_maps_name)
input_file_MBFM = os.path.join('/content','mesonet_inputs', input_file_MBFM_name)
input_file_voxelmorph = os.path.join('/content','mesonet_inputs', input_file_voxelmorph_name)

output_file_atlas_brain = os.path.join('/content','mesonet_outputs', output_file_atlas_brain_name)
output_file_brain_atlas = os.path.join('/content','mesonet_outputs', output_file_brain_atlas_name)
output_file_sensory = os.path.join('/content','mesonet_outputs', output_file_sensory_name)
output_file_MBFM_U_Net = os.path.join('/content','mesonet_outputs', output_file_MBFM_U_Net_name)
output_file_voxelmorph = os.path.join('/content','mesonet_outputs', output_file_voxelmorph_name)

model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', model_name)
u_net_only_model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', u_net_only_model_name)
voxelmorph_model = os.path.join('/content','mesonet_inputs', voxelmorph_model_name)
dlc_config = os.path.join('/content','mesonet_inputs', dlc_model_name, 'config.yaml')

In [22]:
!mkdir '/content/mesonet_outputs'
!mkdir '/content/mesonet_outputs/mesonet_outputs_atlas_brain'
!mkdir '/content/mesonet_outputs/mesonet_outputs_brain_atlas'
!mkdir '/content/mesonet_outputs/mesonet_outputs_sensory'
!mkdir '/content/mesonet_outputs/mesonet_outputs_MBFM_U_Net'
!mkdir '/content/mesonet_outputs/mesonet_outputs_voxelmorph'

mkdir: cannot create directory ‘/content/mesonet_outputs’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_atlas_brain’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_brain_atlas’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_sensory’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_MBFM_U_Net’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_voxelmorph’: File exists


Now that we've told Colab where to find the input and output folders, let's define the configuration file!

# Configure MesoNet

In [23]:
# Set this environment variable to help MesoNet find the git repo location
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

In [24]:
!ls /content/mesonet_outputs/mesonet_outputs_atlas_brain

In [25]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file for each pipeline
import mesonet
## 1. Atlas to brain
# Atlas-to-brain warp with U-Net and DeepLabCut
print('1. Atlas-to-brain warp with U-Net and DeepLabCut')
config_file_atlas_brain = mesonet.config_project(input_file, output_file_atlas_brain, 'test', 
                                                 atlas_to_brain_align=True, use_voxelmorph=False, 
                                                 use_unet=True, use_dlc=True, 
                                                 sensory_match=False, mat_save=False, 
                                                 config=dlc_config, model=model)

## 2. Brain to atlas
# Brain-to-atlas warp with DeepLabCut
print('2. Brain-to-atlas warp with DeepLabCut')
config_file_brain_atlas = mesonet.config_project(input_file, output_file_brain_atlas, 'test', 
                                                 atlas_to_brain_align=False, use_voxelmorph=False, 
                                                 use_unet=False, use_dlc=True, sensory_match=False, 
                                                 mat_save=False, olfactory_check=False, 
                                                 config=dlc_config)

## 3. Atlas to brain + sensory
# Atlas-to-brain warp with U-Net, DeepLabCut, and sensory maps
print('3. Atlas-to-brain warp with U-Net, DeepLabCut, and sensory maps')
config_file_sensory = mesonet.config_project(input_file_sensory_raw, output_file_sensory, 'test',
                                             atlas_to_brain_align=True, use_voxelmorph=False, 
                                             use_unet=True, use_dlc=True, sensory_match=True, 
                                             sensory_path=input_file_sensory_maps, mat_save=False, 
                                             config=dlc_config, model=model)

## 4. MBFM + U-Net
# Motif-based functional maps (MBFMs) with atlas directly applied using U-Net
print('4. Motif-based functional maps (MBFMs) with atlas directly applied using U-Net')
config_file_MBFM_U_Net = mesonet.config_project(input_file_MBFM, output_file_MBFM_U_Net, 'test', 
                                                atlas_to_brain_align=True, use_voxelmorph=False, 
                                                use_unet=True, use_dlc=False, sensory_match=False, 
                                                mat_save=False, mask_generate=False, 
                                                config=dlc_config, model=u_net_only_model)

## 5. VoxelMorph
# Local deformation warp with VoxelMorph and DeepLabCut
print('5. Local deformation warp with VoxelMorph and DeepLabCut')
config_file_voxelmorph = mesonet.config_project(input_file_voxelmorph, output_file_voxelmorph, 'test', 
                                                atlas_to_brain_align=False, use_voxelmorph=True, 
                                                use_unet=True, use_dlc=True, sensory_match=False, mat_save=False, 
                                                config=dlc_config, model=model, 
                                                align_once=True, olfactory_check=True, 
                                                voxelmorph_model=voxelmorph_model)

ModuleNotFoundError: ignored

The config file (by default in each of the output folders) contains information about how MesoNet will run for each pipeline. We'll be using these config files as an input to the last two functions needed to run MesoNet. 

Now, we will run each of the five pipelines in turn:



# Run MesoNet

## Pipeline 1: Atlas-to-brain
Firstly, we will identify the outer edges of the cortex:

### Predict regions

In [ ]:
%cd /content/
mesonet.predict_regions(config_file_atlas_brain)

Next, we will identify and use cortical landmarks to align the atlas to the brain:

### Predict landmarks

In [ ]:
mesonet.predict_dlc(config_file_atlas_brain)

Congratulations, you're all done with this first pipeline! You can now check the outputs in the `mesonet_output_atlas_brain` folder. The segmented brain data can be found in `mesonet_output_atlas_brain/output_overlay`.

The following four pipelines will follow a similar pattern:

## Pipeline 2: Brain-to-atlas
This time, we will directly identify and use cortical landmarks to align the brain to the atlas:

### Predict landmarks

In [ ]:
%cd /content/
mesonet.predict_dlc(config_file_brain_atlas)

You can now check the outputs in the `mesonet_output_brain_atlas` folder. The segmented brain data can be found in `mesonet_output_brain_atlas/output_overlay`.

## Pipeline 3: Atlas-to-Brain + sensory
Now, we return to the Brain-to-Atlas approach while also using peaks of functional activity that are common across animals as a further alignment step.

### Predict regions

In [ ]:
%cd /content/
mesonet.predict_regions(config_file_sensory)

### Predict landmarks

In [ ]:
mesonet.predict_dlc(config_file_sensory)

You can now check the outputs in the `mesonet_output_sensory` folder. The segmented brain data can be found in `mesonet_output_sensory/output_overlay`.

## Pipeline 4: MBFM + U-Net
Our input for this pipeline will be a set of motif-based functional maps (MBFMs) - brain images that summarize patterns of spatio-temporal activity that are common across animals. You can generate these using a MATLAB script running [seqNMF](https://github.com/FeeLab/seqNMF) - such a script is available in `4_Data_code/New_end_to_end_code` on our [OSF repository](https://osf.io/svztu/). We will then use a U-Net model to directly segment the brain image into functional regions - no need for atlas registration here!

### Predict regions

In [ ]:
%cd /content/
mesonet.predict_regions(config_file_MBFM_U_Net)

You can now check the outputs in the `mesonet_output_MBFM_U_Net` folder. The segmented brain data can be found in `mesonet_output_MBFM_U_Net/output_overlay`.

## Pipeline 5: VoxelMorph
Our input for this pipeline will be a raw brain image followed by an MBFM (see Pipeline 4 description for details). We will use VoxelMorph - a local deformation technique - to register the MBFM to an atlas based on a template image in the MesoNet repository.

### Predict landmarks

In [ ]:
mesonet.predict_dlc(config_file_voxelmorph)

You can now check the outputs in the `mesonet_output_voxelmorph` folder. The segmented brain data can be found in `mesonet_output_voxelmorph/output_overlay`.

# Conclusion
These five pipelines can be directly accessed in the graphical user interface (GUI) that is available for MesoNet for ease of use. Furthermore, you can customize your pipeline by changing the options defined in `mesonet.config_project` for the CLI and in the GUI - you can use our [Quick Start Guide](https://github.com/bf777/MesoNet/wiki/Quick-Start-Guide) and [Config File Guide](https://github.com/bf777/MesoNet/wiki/Config-File-Guide) for guidance.